## Recomendação de Produtos com KNN

### Bibliotecas utilizadas

In [1]:
import pandas as pd
import tkinter as tk
from sklearn.neighbors import NearestNeighbors
import numpy as np

### Base de dados

In [2]:
# Carregando o CSV gerado em um DataFrame
df = pd.read_csv('produtos_expandidos.csv')


### Normalizar dados numéricos

In [3]:
# Função para normalizar os dados numéricos
def normalizar(x):
    return (x - np.min(x)) / (np.max(x) - np.min(x))

# Aplicar a normalização aos dados
df[['Preço', 'Avaliação']] = df[['Preço', 'Avaliação']].apply(normalizar)

### Função para recomendação

In [4]:
# Função para encontrar um produto mais similar, considerando múltiplas características
def encontrar_produto_similar(produto):
    # Verificar se o produto existe no DataFrame
    if produto not in df['Produto'].values:
        return ["Produto não encontrado"]

    # Filtrar o DataFrame para produtos da mesma categoria
    categoria_produto = df.loc[df['Produto'] == produto, 'Categoria'].values[0]
    produtos_categoria = df[df['Categoria'] == categoria_produto].copy()

    # Resetar os índices após o filtro
    produtos_categoria.reset_index(drop=True, inplace=True)

    # Obtendo os vetores de características dos produtos
    vetor_caracteristicas = pd.get_dummies(produtos_categoria[['Categoria']])
    vetor_caracteristicas['Preço'] = produtos_categoria['Preço']
    vetor_caracteristicas['Avaliação'] = produtos_categoria['Avaliação']

    # Ponderando as características
    vetor_caracteristicas['Categoria_' + categoria_produto] *= 1.1  # Duplica o peso da categoria do produto

    # Convertendo para valores
    vetor_caracteristicas = vetor_caracteristicas.values

    # Treinando o modelo KNN
    knn_model = NearestNeighbors(n_neighbors=4, algorithm='brute', metric='manhattan')  # n_neighbors = 4 para incluir até 3 produtos similares
    knn_model.fit(vetor_caracteristicas)

    # Encontrando o índice do produto no DataFrame filtrado
    produto_index = produtos_categoria.index[produtos_categoria['Produto'] == produto].tolist()[0]

    # Encontrando os vizinhos mais próximos
    _, indices = knn_model.kneighbors([vetor_caracteristicas[produto_index]])

    # Extraindo os produtos similares (excluindo o próprio produto)
    produtos_similares = [produtos_categoria.iloc[i]['Produto'] for i in indices.flatten() if produtos_categoria.iloc[i]['Produto'] != produto]

    return produtos_similares[:3]  # Retorna apenas 3 produtos similares


### Interface Gráfica

In [5]:
# Criando a interface gráfica
def recomendar_produto():
    produto_comprado = entry_produto.get()
    produtos_recomendados = encontrar_produto_similar(produto_comprado)
    label_resultado.config(text=f"Produtos similares ao '{produto_comprado}': {', '.join(produtos_recomendados)}")

# Configurações da janela
root = tk.Tk()
root.title("Sistema de Recomendação de Produtos Eletrônicos")
root.geometry("400x200")

# Elementos da interface
label_produto = tk.Label(root, text="Produto Comprado:")
label_produto.pack(pady=5)

entry_produto = tk.Entry(root, width=30)
entry_produto.pack(pady=5)

button_recomendar = tk.Button(root, text="Recomendar", command=recomendar_produto)
button_recomendar.pack(pady=5)

label_resultado = tk.Label(root, text="")
label_resultado.pack(pady=5)

# Rodar a interface
root.mainloop()


### Código completo

In [11]:
import pandas as pd
import tkinter as tk
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Carregando o CSV gerado em um DataFrame
df = pd.read_excel('produtos.xlsx')

# Função para normalizar os dados numéricos
def normalizar(x):
    return (x - np.min(x)) / (np.max(x) - np.min(x))

# Aplicar a normalização aos dados
df[['Preço', 'Avaliação']] = df[['Preço', 'Avaliação']].apply(normalizar)

# Função para encontrar um produto mais similar, considerando múltiplas características
def encontrar_produto_similar(produto):
    # Verificar se o produto existe no DataFrame
    if produto not in df['Produto'].values:
        return ["Produto não encontrado"]

    # Filtrar o DataFrame para produtos da mesma categoria
    categoria_produto = df.loc[df['Produto'] == produto, 'Categoria'].values[0]
    produtos_categoria = df[df['Categoria'] == categoria_produto].copy()

    # Resetar os índices após o filtro
    produtos_categoria.reset_index(drop=True, inplace=True)

    # Obtendo os vetores de características dos produtos
    vetor_caracteristicas = pd.get_dummies(produtos_categoria[['Categoria']])
    vetor_caracteristicas['Preço'] = produtos_categoria['Preço']
    vetor_caracteristicas['Avaliação'] = produtos_categoria['Avaliação']

    # Ponderando as características
    vetor_caracteristicas['Categoria_' + categoria_produto] *= 2  # Duplica o peso da categoria do produto

    # Convertendo para valores
    vetor_caracteristicas = vetor_caracteristicas.values

    # Treinando o modelo KNN
    knn_model = NearestNeighbors(n_neighbors=4, algorithm='brute', metric='manhattan')  # n_neighbors = 4 para incluir até 3 produtos similares
    knn_model.fit(vetor_caracteristicas)

    # Encontrando o índice do produto no DataFrame filtrado
    produto_index = produtos_categoria.index[produtos_categoria['Produto'] == produto].tolist()[0]

    # Encontrando os vizinhos mais próximos
    _, indices = knn_model.kneighbors([vetor_caracteristicas[produto_index]])

    # Extraindo os produtos similares (excluindo o próprio produto)
    produtos_similares = [produtos_categoria.iloc[i]['Produto'] for i in indices.flatten() if produtos_categoria.iloc[i]['Produto'] != produto]

    return produtos_similares[:3]  # Retorna apenas 3 produtos similares

# Criando a interface gráfica
def recomendar_produto():
    produto_comprado = entry_produto.get()
    produtos_recomendados = encontrar_produto_similar(produto_comprado)
    label_resultado.config(text=f"Produtos similares ao '{produto_comprado}': {', '.join(produtos_recomendados)}")

# Configurações da janela
root = tk.Tk()
root.title("Sistema de Recomendação de Produtos Eletrônicos")
root.geometry("400x200")

# Elementos da interface
label_produto = tk.Label(root, text="Produto Comprado:")
label_produto.pack(pady=5)

entry_produto = tk.Entry(root, width=30)
entry_produto.pack(pady=5)

button_recomendar = tk.Button(root, text="Recomendar", command=recomendar_produto)
button_recomendar.pack(pady=5)

label_resultado = tk.Label(root, text="")
label_resultado.pack(pady=5)

# Rodar a interface
root.mainloop()
